In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import re

In [3]:
a = np.load('../initials/beta0.5_nu1e-10_29dim.npy')
# plt.plot(abs(a[0, :]), abs(a[1, :]))
a

array([ 4.99989997e-001+4.99992628e-001j,
        3.93937589e-005-2.37842167e-005j,
       -1.06333771e-005-4.48296526e-005j,
       -9.05665633e-006+3.57573185e-006j,
        3.58846651e-006+8.68882329e-006j,
       -2.29443388e-006+3.59260752e-007j,
        6.69337618e-006-9.33141627e-006j,
       -8.90676239e-006+4.73326220e-007j,
        3.33290752e-006-9.73094887e-006j,
       -2.46299888e-006-8.89722022e-006j,
       -8.75853382e-007+3.67631230e-006j,
        1.76678761e-006+7.28178306e-006j,
        5.20686714e-006+2.44713653e-006j,
       -6.74470197e-006+3.05989939e-006j,
       -5.91389794e-006+4.08342395e-006j,
        1.57931616e-006-4.27239331e-006j,
       -1.71428703e-006-2.85554470e-006j,
        1.19772907e-006+7.82886246e-007j,
        2.21539526e-007+2.70445426e-008j,
       -4.24480876e-009-5.18524394e-009j,
        1.25803844e-011+7.76287233e-012j,
        4.74956792e-016+6.45814982e-017j,
       -1.07879184e-023-1.31294639e-023j,
        8.28486301e-036+5.10788676

## error_growthのnpyの結合

In [3]:
# "../error_growth/assemble/" から.npyファイルを全て読み込む
directory = "../error_growth/assemble/"
files = os.listdir(directory)
files_file = [f for f in files if os.path.isfile(os.path.join(directory, f))]
files_npy = [f for f in files_file if f[-4:] == '.npy']

# ファイル名からパラメータを取得
beta, nu, t, t_initial, dt = re.search(r'beta(\d+\.\d+)nu(\d+e-\d+)t(\d+)initial(\d+)dt(\d+\.\d+)', files_npy[0]).groups()
perturbed_dim = re.search(r'perturbed_dim(\d+)', files_npy[0]).group(1)
repetition = 0
time = np.load(directory + files_npy[0])[1,]
average = np.zeros_like(time)

# ファイルを読み込んで平均を取る
for i in range(len(files_npy)):
    beta_local, nu_local, t_local, t_initial_local, dt_local = re.search(r'beta(\d+\.\d+)nu(\d+e-\d+)t(\d+)initial(\d+)dt(\d+\.\d+)', files_npy[i]).groups()
    perturbed_dim_local = re.search(r'perturbed_dim(\d+)', files_npy[i]).group(1)
    # パラメータが一致しているか確認
    if beta_local != beta or nu_local != nu or t_local != t or t_initial_local != t_initial or dt_local != dt or perturbed_dim_local != perturbed_dim:
        print("Parameter is not matched")
        continue
    repetition_local = int(re.search(r'repeat(\d+)', files_npy[i]).group(1))
    repetition += repetition_local
    average += np.load(directory + files_npy[i])[0, ] * repetition_local
    
average /= repetition
error_growth = (np.log(average[2:, ]) - np.log(average[:-2, ])) / (2*float(dt))

arr = np.array([average[1:-1], error_growth, time[1:-1]])
# 保存
np.save("../error_growth/linear_beta" + beta + "nu" + nu + "t" + t + "initial" + t_initial + "dt" + dt + "repeat" + str(repetition) + "perturbed_dim" + perturbed_dim + ".npy", arr)
    